# Topic and word list from a working paper by Sophie Stone of the Dartmouth College Economics Department



## https://www.sbert.net/docs/usage/semantic_textual_similarity.html

In [ ]:
# Dependencies
# !pip install "calcbench-api-client[Pandas, Backoff, BeautifulSoup]" sentence-transformers tqdm qgrid ipywidgets

In [3]:
import warnings
from collections import Counter

import pandas as pd
import itertools
from tqdm.notebook import tqdm
import qgrid

import calcbench as cb


cb.enable_backoff()
tqdm.pandas(desc="progress")

In [66]:
categories = {
    "environmental": """biodiversity, carbon, carbon negative, carbon neutral, carbon zero, clean-up, clean, clean tech, climate change, climate positive, contamination, discharge, emission, energy-efficient, environmental, environmental risk, fuel, fuel efficiency, green, greenhouse gas, hazardous, low carbon, natural resource, net-zero emission, pollution, remediation, sustainability, sustainable, toxic, waste, water, zero carbon, zero net carbon""",
    "social": """accident, antiracism, consumer protection, customer privacy, employee relation, equal, equal pay, equity, gender equality, health, human right, justice, labor relation, labor standard, racial equity, racial awareness, racial justice, working condition""",
    "governance": """advocacy, antitrust, board independence, code of ethic, compensation, corporate culture, corporate governance, corruption, governance risk, justice, political lobbying, scandal, shareholder right, stability, stewardship, transparency""",
}

category_embeddings = {
    category: set(w.strip() for w in category_words.split(","))
    for (category, category_words) in categories.items()
}

In [67]:
category_embeddings

{'environmental': {'biodiversity',
  'carbon',
  'carbon negative',
  'carbon neutral',
  'carbon zero',
  'clean',
  'clean tech',
  'clean-up',
  'climate change',
  'climate positive',
  'contamination',
  'discharge',
  'emission',
  'energy-efficient',
  'environmental',
  'environmental risk',
  'fuel',
  'fuel efficiency',
  'green',
  'greenhouse gas',
  'hazardous',
  'low carbon',
  'natural resource',
  'net-zero emission',
  'pollution',
  'remediation',
  'sustainability',
  'sustainable',
  'toxic',
  'waste',
  'water',
  'zero carbon',
  'zero net carbon'},
 'social': {'accident',
  'antiracism',
  'consumer protection',
  'customer privacy',
  'employee relation',
  'equal',
  'equal pay',
  'equity',
  'gender equality',
  'health',
  'human right',
  'justice',
  'labor relation',
  'labor standard',
  'racial awareness',
  'racial equity',
  'racial justice',
  'working condition'},
 'governance': {'advocacy',
  'antitrust',
  'board independence',
  'code of ethic'

In [10]:
with tqdm() as progress_bar:
    disclosures = cb.document_dataframe(
        company_identifiers=cb.tickers(index="DJIA"),
        disclosure_names=["ManagementsDiscussionAndAnalysis", "RiskFactors"],
        all_history=True,
        period_type="annual",
        progress_bar=progress_bar,
        # entire_universe=True,
    )

0it [00:00, ?it/s]

In [11]:
def get_contents(d):
    try:
        return d.get_contents_text()
    except Exception as e:
        print(f"Exception getting {d} \n{e}")


disclosure_contents = disclosures.progress_applymap(
    get_contents, na_action="ignore"
).fillna("")

progress:   0%|          | 0/900 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
disclosure_contents = pd.read_pickle("disclosure_contents.pkl")

In [14]:
disclosure_embeddings = disclosure_contents.progress_applymap(
    lambda c: Counter(c.split())
)

progress:   0%|          | 0/900 [00:00<?, ?it/s]

In [44]:
def distance(column_embeddings: Counter, category_embeddings: set):
    print(column_embeddings)
    return sum(count for word, count in column_embeddings.items() if word in category_embeddings)

In [51]:
distances = []
for _, category_words in tqdm(categories.items()):
    category_distances = disclosure_embeddings.progress_applymap(
        lambda word_counts: sum(count for word, count in word_counts.items() if word in category_words)
    )
    distances.append(category_distances)
esg_distances = pd.concat(
    distances, axis=1, keys=[category for category, _ in categories.items()]
)

  0%|          | 0/3 [00:00<?, ?it/s]

progress:   0%|          | 0/900 [00:00<?, ?it/s]

progress:   0%|          | 0/900 [00:00<?, ?it/s]

progress:   0%|          | 0/900 [00:00<?, ?it/s]

In [70]:
esg_distances.environmental / disclosure_embeddings.progress_applymap(lambda d: sum(d.values()))

progress:   0%|          | 0/900 [00:00<?, ?it/s]

disclosure_type_name ManagementsDiscussionAndAnalysis                      \
ticker                                           AAPL      AMGN       AXP   
period                                                                      
2007                                              NaN       NaN       NaN   
2008                                              NaN       NaN       NaN   
2009                                         0.095056  0.113332  0.100589   
2010                                         0.095013  0.112136  0.102545   
2011                                         0.096849  0.113288  0.098601   
2012                                         0.099883  0.120983  0.100678   
2013                                         0.097571  0.117795  0.098635   
2014                                         0.100646  0.112709  0.113783   
2015                                         0.099127  0.114365  0.108430   
2016                                         0.098814  0.111216  0.109207   
2017                                         0.100578  0.114322  0.109288   
2018                                         0.105013  0.113566  0.107784   
2019                                         0.096217  0.112656  0.109706   
2020                                         0.099432  0.117454  0.116281   
2021                                              NaN       NaN       NaN   

disclosure_type_name                                                    \
ticker                      BA       CAT       CRM      CSCO       CVX   
period                                                                   
2007                       NaN       NaN       NaN       NaN       NaN   
2008                       NaN       NaN       NaN       NaN       NaN   
2009                  0.103651  0.095581       NaN       NaN  0.138889   
2010                  0.104647  0.094241  0.100024  0.114542  0.138889   
2011                  0.101509  0.069583  0.105519  0.114159  0.156250   
2012                  0.106739  0.100709  0.109438  0.114235  0.098191   
2013                  0.107302       NaN  0.112211  0.121479  0.073770   
2014                  0.107419  0.108975  0.113343  0.122921  0.073171   
2015                  0.112121  0.110315  0.117067  0.121974  0.078261   
2016                  0.108193  0.111057  0.114995  0.119294  0.078261   
2017                  0.103393  0.113583  0.118536  0.118749  0.078261   
2018                  0.103850  0.107775  0.123616  0.115632  0.078947   
2019                  0.110518  0.099139  0.130948  0.110087  0.078947   
2020                  0.118656  0.094678  0.127738  0.108675  0.078947   
2021                       NaN       NaN  0.131655       NaN       NaN   

disclosure_type_name                                                    \
ticker                     DIS       DOW        GS        HD       HON   
period                                                                   
2007                       NaN       NaN       NaN       NaN       NaN   
2008                       NaN       NaN       NaN       NaN       NaN   
2009                  0.100254       NaN  0.114490  0.110978  0.113894   
2010                  0.094739       NaN  0.112083  0.114489  0.109577   
2011                  0.100617       NaN  0.106948  0.120953  0.107907   
2012                  0.097699       NaN  0.106270  0.125059  0.110492   
2013                  0.102605       NaN  0.110161  0.121982  0.109485   
2014                  0.108294       NaN  0.116484  0.127275  0.108926   
2015                  0.108942       NaN  0.114134  0.125208  0.105367   
2016                  0.113500       NaN  0.113174  0.123461  0.100978   
2017                  0.114003  0.116348  0.114342  0.122682  0.098442   
2018                  0.108554  0.114446  0.114883  0.127209  0.098251   
2019                  0.109557  0.113797  0.114833  0.116723  0.107524   
2020                  0.110226  0.109095  0.112392  0.129710  0.117494   
2021                  